# Little Lemon MySQL

In [84]:
import mysql.connector as connector
from mysql.connector.pooling import MySQLConnectionPool, PoolError
from mysql.connector import Error

## Setup

### Create Database

In [2]:
connection=connector.connect(user="root", password ="")

In [3]:
cursor = connection.cursor()

In [4]:
cursor.execute("CREATE DATABASE little_lemon_db")

In [5]:
cursor.execute("USE little_lemon_db")

### Create Tables

In [6]:
#MenuItems table
create_menuitem_table = """CREATE TABLE MenuItems (
ItemID INT AUTO_INCREMENT,
Name VARCHAR(200),
Type VARCHAR(100),
Price INT,
PRIMARY KEY (ItemID)
);"""

In [7]:
create_menu_table = """CREATE TABLE Menus (
MenuID INT,
ItemID INT,
Cuisine VARCHAR(100),
PRIMARY KEY (MenuID,ItemID)
);"""

In [11]:
create_booking_table = """CREATE TABLE Bookings (
BookingID INT AUTO_INCREMENT,
TableNo INT,
GuestFirstName VARCHAR(100) NOT NULL,
GuestLastName VARCHAR(100) NOT NULL,
BookingSlot TIME NOT NULL,
EmployeeID INT,
PRIMARY KEY (BookingID)
);"""

In [9]:
create_orders_table = """CREATE TABLE Orders (
OrderID INT,
TableNo INT,
MenuID INT,
BookingID INT,
BillAmount INT,
Quantity INT,
PRIMARY KEY (OrderID,TableNo)
);"""

In [10]:
create_employees_table = """CREATE TABLE Employees (
EmployeeID INT AUTO_INCREMENT PRIMARY KEY,
Name VARCHAR (255),
Role VARCHAR (100),
Address VARCHAR (255),
Contact_Number INT,
Email VARCHAR (255),
Annual_Salary VARCHAR (100)
);"""

In [19]:
# Create MenuItems table
cursor.execute(create_menuitem_table)
# Create Menu table
cursor.execute(create_menu_table)
# Create Bookings table
cursor.execute(create_booking_table)
# Create Orders table
cursor.execute(create_orders_table)
# Create Employees table
cursor.execute(create_employees_table)

### Insert Data

In [17]:
insert_menuitems="""
INSERT INTO MenuItems (ItemID, Name, Type, Price)
VALUES
(1, 'Olives','Starters',5),
(2, 'Flatbread','Starters', 5),
(3, 'Minestrone', 'Starters', 8),
(4, 'Tomato bread','Starters', 8),
(5, 'Falafel', 'Starters', 7),
(6, 'Hummus', 'Starters', 5),
(7, 'Greek salad', 'Main Courses', 15),
(8, 'Bean soup', 'Main Courses', 12),
(9, 'Pizza', 'Main Courses', 15),
(10, 'Greek yoghurt','Desserts', 7),
(11, 'Ice cream', 'Desserts', 6),
(12, 'Cheesecake', 'Desserts', 4),
(13, 'Athens White wine', 'Drinks', 25),
(14, 'Corfu Red Wine', 'Drinks', 30),
(15, 'Turkish Coffee', 'Drinks', 10),
(16, 'Turkish Coffee', 'Drinks', 10),
(17, 'Kabasa', 'Main Courses', 17);"""

insert_menu="""
INSERT INTO Menus (MenuID,ItemID,Cuisine)
VALUES
(1, 1, 'Greek'),
(1, 7, 'Greek'),
(1, 10, 'Greek'),
(1, 13, 'Greek'),
(2, 3, 'Italian'),
(2, 9, 'Italian'),
(2, 12, 'Italian'),
(2, 15, 'Italian'),
(3, 5, 'Turkish'),
(3, 17, 'Turkish'),
(3, 11, 'Turkish'),
(3, 16, 'Turkish');"""

insert_bookings="""
INSERT INTO Bookings (BookingID, TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(1, 12, 'Anna','Iversen','19:00:00',1),
(2, 12, 'Joakim', 'Iversen', '19:00:00', 1),
(3, 19, 'Vanessa', 'McCarthy', '15:00:00', 3),
(4, 15, 'Marcos', 'Romero', '17:30:00', 4),
(5, 5, 'Hiroki', 'Yamane', '18:30:00', 2),
(6, 8, 'Diana', 'Pinto', '20:00:00', 5);"""

insert_orders="""
INSERT INTO Orders (OrderID, TableNo, MenuID, BookingID, Quantity, BillAmount)
VALUES
(1, 12, 1, 1, 2, 86),
(2, 19, 2, 2, 1, 37),
(3, 15, 2, 3, 1, 37),
(4, 5, 3, 4, 1, 40),
(5, 8, 1, 5, 1, 43);"""

insert_employees = """
INSERT INTO employees (EmployeeID, Name, Role, Address, Contact_Number, Email, Annual_Salary)
VALUES
(01,'Mario Gollini','Manager','724, Parsley Lane, Old Town, Chicago, IL',351258074,'Mario.g@littlelemon.com','$70,000'),
(02,'Adrian Gollini','Assistant Manager','334, Dill Square, Lincoln Park, Chicago, IL',351474048,'Adrian.g@littlelemon.com','$65,000'),
(03,'Giorgos Dioudis','Head Chef','879 Sage Street, West Loop, Chicago, IL',351970582,'Giorgos.d@littlelemon.com','$50,000'),
(04,'Fatma Kaya','Assistant Chef','132  Bay Lane, Chicago, IL',351963569,'Fatma.k@littlelemon.com','$45,000'),
(05,'Elena Salvai','Head Waiter','989 Thyme Square, EdgeWater, Chicago, IL',351074198,'Elena.s@littlelemon.com','$40,000'),
(06,'John Millar','Receptionist','245 Dill Square, Lincoln Park, Chicago, IL',351584508,'John.m@littlelemon.com','$35,000');"""

In [20]:
# Populate MenuItems table
cursor.execute(insert_menuitems)
connection.commit()

# Populate MenuItems table
cursor.execute(insert_menu)
connection.commit()

# Populate Bookings table
cursor.execute(insert_bookings)
connection.commit()

# Populate Orders table
cursor.execute(insert_orders)
connection.commit()

# Populate Employees table
cursor.execute(insert_employees)
connection.commit()

## Implement a connection pool and stored procedure


In [69]:
dbconfig={"database":"little_lemon_db", "user":"root", "password":""}

In [70]:
try:
    little_lemon_pool = MySQLConnectionPool(
        pool_name="little_lemon_pool",
        pool_size=2,
        **dbconfig
    )
    print("Connection pool created successfully")
except Error as err:
    print(f"Error: {err}")

Connection pool created successfully


In [71]:
example_query = "SELECT * FROM Bookings"

In [57]:
def execute_query_on_pool(query, pool=little_lemon_pool):
    conn=pool.get_connection()
    if conn.is_connected():
        cursor=conn.cursor()
        print(f"Connection established to {conn.connection_id}")
        cursor.execute(query)
        result=cursor.fetchall()
        print(result)
        conn.close()

In [58]:
execute_query_on_pool(example_query)

Connection established to 15
[(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1), (2, 12, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 1), (3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3), (4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4), (5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2), (6, 8, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5)]


In [72]:
peak_hours_procedure = """
CREATE PROCEDURE peak_hours()
BEGIN
    SELECT
        HOUR(BookingSlot) as BookingHour,
        COUNT(*) AS BookingCount
    FROM Bookings
    GROUP BY BookingHour
    ORDER BY BookingCount DESC;
END
"""

In [75]:
# Step 2: Execute the procedure creation
try:
    conn = little_lemon_pool.get_connection()
    cursor = conn.cursor()
    cursor.execute("DROP PROCEDURE IF EXISTS peak_hours")
    cursor.execute(peak_hours_procedure)
    print("Procedure created successfully")

    # Step 3 & 4: Call procedure and fetch results
    cursor.callproc("peak_hours")
    results = next(cursor.stored_results())
    dataset = results.fetchall()

    # Step 5: Get column names
    column_names = [desc[0] for desc in results.description]

    # Step 6: Print column names
    print("\nColumns:", column_names)

    # Step 7: Print results
    print("\nResults:")
    for row in dataset:
        print(f"Hour: {row[0]}, Number of Bookings: {row[1]}")

except Error as err:
    print(f"Error: {err}")
finally:
    if 'connection' in locals() and conn.is_connected():
        cursor.close()
        conn.close()

Procedure created successfully

Columns: ['BookingHour', 'BookingCount']

Results:
Hour: 19, Number of Bookings: 2
Hour: 15, Number of Bookings: 1
Hour: 17, Number of Bookings: 1
Hour: 18, Number of Bookings: 1
Hour: 20, Number of Bookings: 1


In [78]:
guest_status_procedure = """
CREATE PROCEDURE GuestStatus()
BEGIN
    SELECT
        CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS GuestName,
        CASE
            WHEN (Employees.Role = 'Manager') OR (Employees.Role = 'Assistant Manager') THEN 'Ready to pay'
            WHEN (Employees.Role = 'Head Chef') THEN 'Ready to serve'
            WHEN (Employees.Role = 'Assistant Chef') THEN 'Preparing Order'
            WHEN (Employees.Role = 'Head Waiter') THEN 'Order served'
            ELSE 'Pending'
        END AS Status
    FROM Bookings
    LEFT JOIN Employees on Bookings.EmployeeID = Employees.EmployeeID;
END
"""

In [79]:
try:
    conn = little_lemon_pool.get_connection()
    cursor = conn.cursor()
    cursor.execute("DROP PROCEDURE IF EXISTS GuestStatus")
    cursor.execute(guest_status_procedure)
    print("Procedure created successfully")

    # Step 3 & 4: Call procedure and fetch results
    cursor.callproc("GuestStatus")
    results = next(cursor.stored_results())
    dataset = results.fetchall()

    # Step 5: Get column names
    column_names = [desc[0] for desc in results.description]

    # Step 6: Print column names
    print("\nColumns:", column_names)

    # Step 7: Print results
    print("\nResults:")
    for row in dataset:
        print(row)

except Error as err:
    print(f"Error: {err}")
finally:
    if 'connection' in locals() and conn.is_connected():
        cursor.close()
        conn.close()

Procedure created successfully

Columns: ['GuestName', 'Status']

Results:
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')


## Final Assessment

### Task 1

In [80]:
dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": ""
}

In [81]:
try:
    pool_b = MySQLConnectionPool(
        pool_name="pool_b",
        pool_size=2,
        **dbconfig
    )
    print("Connection pool created successfully")
except Error as err:
    print(f"Error: {err}")

Connection pool created successfully


### Task 2

In [82]:
# Insert query template
insert_booking_query = """
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES (%s, %s, %s, %s, %s)
"""

# Guest data
guests = [
    (8, "Anees", "Java", "18:00:00", 6),
    (5, "Bald", "Vin", "19:00:00", 6),
    (12, "Jay", "Kon", "19:30:00", 6)
]

In [85]:
# List to store connections
conns = []

# Get connections and insert bookings
try:
    # Get connections for each guest
    for _ in range(3):
        conn = pool_b.get_connection()
        conns.append(conn)
        cursor = conn.cursor()
        guest_data = guests[len(conns)-1]
        cursor.execute(insert_booking_query, guest_data)
        conn.commit()
        print(f"Booking added for {guest_data[1]} {guest_data[2]}")
        cursor.close()
except PoolError as err:
    print(f"Error: {err}")
finally:
    # Return connections to pool
    for connection in conns:
        try:
            connection.close()
            print("Connection returned to pool")
        except Error as err:
            print(f"Error returning connection to pool: {err}")


Booking added for Anees Java
Booking added for Bald Vin
Error: Failed getting connection; pool exhausted
Connection returned to pool
Connection returned to pool


### Task 3

In [86]:
manager_query = """
SELECT Name, EmployeeID
FROM Employees
WHERE Role = 'Manager';
"""

# Query 2: Highest paid employee
highest_salary_query = """
SELECT
    Name,
    Role,
    Annual_Salary
FROM Employees
ORDER BY Annual_Salary DESC
LIMIT 1;
"""

# Query 3: Evening bookings count
evening_bookings_query = """
SELECT COUNT(*) as BookingsCount
FROM Bookings
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';
"""

# Query 4: Guests waiting with receptionist
waiting_guests_query = """
SELECT
    BookingID,
    CONCAT(GuestFirstName, ' ', GuestLastName) as GuestName,
    BookingSlot
FROM Bookings b
JOIN Employees e ON b.EmployeeID = e.EmployeeID
WHERE e.Role = 'Receptionist'
ORDER BY BookingSlot;
"""

In [87]:
# Execute all queries
try:
    conn = pool_b.get_connection()
    cursor = conn.cursor()

    # Execute and print manager info
    cursor.execute(manager_query)
    print("\nManager Information:")
    for row in cursor.fetchall():
        print(f"Name: {row[0]}, EmployeeID: {row[1]}")

    # Execute and print highest salary info
    cursor.execute(highest_salary_query)
    print("\nHighest Paid Employee:")
    for row in cursor.fetchall():
        print(f"Name: {row[0]}, Role: {row[1]}, Salary: {row[2]}")

    # Execute and print evening bookings count
    cursor.execute(evening_bookings_query)
    print("\nNumber of Evening Bookings (18:00-20:00):")
    print(cursor.fetchone()[0])

    # Execute and print waiting guests
    cursor.execute(waiting_guests_query)
    print("\nGuests Waiting with Receptionist:")
    print("BookingID | Guest Name | Booking Time")
    print("-" * 40)
    for row in cursor.fetchall():
        print(f"{row[0]} | {row[1]} | {row[2]}")

except Error as err:
    print(f"Error: {err}")
finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()


Manager Information:
Name: Mario Gollini, EmployeeID: 1

Highest Paid Employee:
Name: Mario Gollini, Role: Manager, Salary: $70,000

Number of Evening Bookings (18:00-20:00):
8

Guests Waiting with Receptionist:
BookingID | Guest Name | Booking Time
----------------------------------------
7 | Anees Java | 18:00:00
9 | Anees Java | 18:00:00
8 | Bald Vin | 19:00:00
10 | Bald Vin | 19:00:00


### Task 4

In [88]:
basic_sales_procedure = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
    SELECT
        SUM(BillAmount) as TotalSales,
        AVG(BillAmount) as AverageSale,
        MIN(BillAmount) as MinimumBill,
        MAX(BillAmount) as MaximumBill
    FROM Orders;
END
"""

In [90]:
try:
    conn = pool_b.get_connection()
    cursor = conn.cursor()

    # Create procedure
    cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport")
    cursor.execute(basic_sales_procedure)
    print("Procedure created successfully")

    # Call procedure and fetch results
    cursor.callproc("BasicSalesReport")
    results = next(cursor.stored_results())
    dataset = results.fetchall()

    # Print results
    print("\nSales Report:")
    print(f"Total Sales: ${dataset[0][0]}")
    print(f"Average Sale: ${dataset[0][1]:.2f}")
    print(f"Minimum Bill: ${dataset[0][2]}")
    print(f"Maximum Bill: ${dataset[0][3]}")
except Error as err:
    print(f"Error: {err}")
finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()

Procedure created successfully

Sales Report:
Total Sales: $243
Average Sale: $48.60
Minimum Bill: $37
Maximum Bill: $86


### Task 5

In [91]:
upcoming_bookings_query = """
SELECT
    b.BookingSlot,
    CONCAT(b.GuestFirstName, ' ', b.GuestLastName) as Guest_Name,
    CONCAT('Assigned to: ', e.Name, ' [', e.Role, ']') as Employee_Info
FROM Bookings b
JOIN Employees e ON b.EmployeeID = e.EmployeeID
ORDER BY b.BookingSlot ASC
LIMIT 3
"""

In [92]:
try:
    # Get connection from pool
    conn = pool_b.get_connection()
    # Create buffered cursor
    cursor = conn.cursor(buffered=True)
    # Execute query
    cursor.execute(upcoming_bookings_query)
    results = cursor.fetchall()
    # Print results
    print("\nUpcoming Bookings:")
    print("-" * 50)
    for booking in results:
        print(f"\n{booking[0]}")
        print(f"{booking[1]}")
        print(f"{booking[2]}")
except Error as err:
    print(f"Error: {err}")
finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print("\nConnection returned to pool")


Upcoming Bookings:
--------------------------------------------------

15:00:00
Vanessa McCarthy
Assigned to: Giorgos Dioudis [Head Chef]

17:30:00
Marcos Romero
Assigned to: Fatma Kaya [Assistant Chef]

18:00:00
Anees Java
Assigned to: John Millar [Receptionist]

Connection returned to pool
